# Eval

In [68]:
import pickle
import pandas as pd
from ast import literal_eval

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

with open(f"data/model_data/dev.pkl", 'rb') as fid:
    df = pickle.load(fid)
    
#with open(f"data/model_data/dev-hard_label-predictions.txt", 'r') as fid:
#    preds = fid.readlines()
#df['hard_label_preds'] = [int(x.strip()) for x in preds]

label_fns = [
    'isa_pattern', 
    'suchas_pattern',
    'including_pattern',
    'called_pattern',
    'especially_pattern',
    'appo_pattern', 
    'other_pattern', 
    'are_pattern',
    'term_modifier',
    'term_subset',
    'whichis_pattern'
]
df.label_model_labels = [tuple(x) for x in df.label_model_labels]
df['label_fn_label'] = df.apply(lambda row: max([row[c] for c in label_fns]), axis=1)

In [69]:
def flip_tp(x, label):
    if x[label] == 1:
        return x['term_pair']
    else:
        return (x['term_pair'][1], x['term_pair'][0])
    
def rev_tps(tps):
    return set([(x[1], x[0]) for x in tps])

def term_eval(df, pred_col, label_col):
    df[pred_col] = df[pred_col].replace(-1, 0)
    
    label_tps = df[df[label_col] > 0][['term_pair', label_col]]
    label_tps = set(label_tps.apply(lambda x: flip_tp(x, label_col), axis=1))
    pred_tps = df[df[pred_col] > 0][['term_pair', pred_col]]
    pred_tps = set(pred_tps.apply(lambda x: flip_tp(x, pred_col), axis=1))
    
    fp = pred_tps - label_tps 
    tp = pred_tps.intersection(label_tps)
    fn = label_tps - pred_tps
    
    print(f"Term Pair Precision: {(len(tp)) / (len(fp) + len(tp))}")
    print(f"Term Pair Recall: {(len(tp)) / (len(fn) + len(tp))}")
    
    return {'false_positives': fp, 'true_positives': tp, 'false_negatives': fn}

In [75]:
kb_ec = term_eval(df, 'kb_bio101_ds_taxonomy', 'gold_label')

Term Pair Precision: 0.6842105263157895
Term Pair Recall: 0.3611111111111111


In [71]:
lfn_ec = term_eval(df, 'label_fn_label', 'gold_label')

Term Pair Precision: 0.7009345794392523
Term Pair Recall: 0.6944444444444444


In [74]:
lfn_ec['false_negatives']

{('actin', 'globular protein'),
 ('allele', 'gene'),
 ('alpha tubulin', 'globular protein'),
 ('animal', 'heterotroph'),
 ('beta tubulin', 'globular protein'),
 ('blood type', 'trait'),
 ('cadherin', 'protein'),
 ('chloroplast', 'plastid'),
 ('chromatin', 'complex'),
 ('collagen', 'protein'),
 ('cyclin', 'molecule'),
 ('dna', 'genetic material'),
 ('dna', 'hereditary material'),
 ('electrocardiogram', 'test'),
 ('epithelial cell', 'animal cell'),
 ('keratin', 'fibrous protein'),
 ('membrane', 'plasma membrane'),
 ('microbe', 'organism'),
 ('myosin', 'motor protein'),
 ('nucleus', 'membrane bind organelle'),
 ('phagocytosis', 'endocytosis'),
 ('phloem', 'vascular tissue'),
 ('phospholipid bilayer', 'membrane'),
 ('plant cell', 'eukaryotic cell'),
 ('plasmid', 'dna'),
 ('protein', 'molecule'),
 ('ribosome', 'cellular structure'),
 ('sarcoplasmic reticulum', 'ser'),
 ('septum', 'cell wall'),
 ('sex cell', 'human cell'),
 ('sperm', 'gamete'),
 ('x chromosome', 'sex chromosome'),
 ('xylem',

In [57]:
#fp_rev = set([(x[1], x[0]) for x in fp])
df[(df.term_pair.isin(label_fn_error_classes['false_positives'])) & (df.label_fn_label > 0)].sample(10)

,textbook,chapter,sentence_id,text,tokens,term1,term1_location,term2,term2_location,term_pair,isa_pattern,suchas_pattern,including_pattern,called_pattern,especially_pattern,appo_pattern,other_pattern,are_pattern,whichis_pattern,term_part_of_speech,term_dep_role,term_modifier,term_subset,kb_bio101_ds_taxonomy,kb_bio101_ds_negative,nsubj_pattern,majority_vote,label_model_labels,gold_label,hard_label,soft_label,label_fn_label
3274,OpenStax Biology 2e,10,4.2,"In humans, the frequency of cell turnover ranges from a few hours in early embryonic development, to an average of two to five days for epithelial cells, and to an entire human lifetime spent in G 0 by specialized cells, such as cortical neurons or cardiac muscle cells.","[In, humans, ,, the, frequency, of, cell, turnover, ranges, from, a, few, hours, in, early, embryonic, development, ,, to, an, average, of, two, to, five, days, for, epithelial, cells, ,, and, to, an, entire, human, lifetime, spent, in, G, 0, , by, specialized, cells, ,, such, as, cortical, neurons, or, cardiac, muscle, cells, .]",epithelial cell,"(27, 29)",cell,"(43, 44)","(epithelial cell, cell)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,0,-1,-1,"(0.00011379332245762893, 0.712603418374233, 0.2872827883033095)",0,0,"[0.00011379332245762893, 0.712603418374233, 0.2872827883033095]",1
2739,OpenStax Biology 2e,10,3.21,"The two centrosomes of homologous chromosomes will give rise to the mitotic spindle , the apparatus that orchestrates the movement of chromosomes during mitosis.","[The, two, centrosomes, of, homologous, chromosomes, will, give, rise, to, the, , mitotic, spindle, ,, the, apparatus, that, orchestrates, the, movement, of, chromosomes, during, mitosis, .]",homologous chromosome,"(4, 6)",chromosome,"(22, 23)","(homologous chromosome, chromosome)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,0,-1,-1,"(0.00011379332245762893, 0.712603418374233, 0.2872827883033095)",0,0,"[0.00011379332245762893, 0.712603418374233, 0.2872827883033095]",1
3654,OpenStax Biology 2e,10,4.64,"Although the cyclins are the main regulatory molecules that determine the forward momentum of the cell cycle, there are several other mechanisms that fine-tune the progress of the cycle with negative, rather than positive, effects.","[Although, the, cyclins, are, the, main, regulatory, molecules, that, determine, the, forward, momentum, of, the, cell, cycle, ,, there, are, several, other, mechanisms, that, fine, -, tune, the, progress, of, the, cycle, with, negative, ,, rather, than, positive, ,, effects, .]",cell cycle,"(15, 17)",cycle,"(31, 32)","(cell cycle, cycle)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,0,-1,-1,"(0.00011379332245762893, 0.712603418374233, 0.2872827883033095)",0,0,"[0.00011379332245762893, 0.712603418374233, 0.2872827883033095]",1
1125,OpenStax Biology 2e,4,4.138,Chloroplasts are plant cell organelles that carry out photosynthesis.,"[Chloroplasts, , are, plant, cell, organelles, that, carry, out, photosynthesis, .]",chloroplast,"(0, 1)",organelle,"(5, 6)","(chloroplast, organelle)",-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0,-1,-1,"(0.00019213144953030562, 0.7192684587504488, 0.28053940980002084)",0,0,"[0.00019213144953030562, 0.7192684587504488, 0.28053940980002084]",1
463,OpenStax Biology 2e,4,4.25,"People with celiac disease have an immune response to gluten, which is a protein in wheat, barley, and rye.","[People, with, celiac, disease, have, an, immune, response, to, gluten, ,, which, is, a, protein, in, wheat, ,, barley, ,, and, rye, .]",immune response,"(6, 8)",protein,"(14, 15)","(immune response, protein)",-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,0,-1,-1,"(0.0037905063106579446, 0.7045581177477463, 0.2916513759415957)",0,0,"[0.0037905063106579446, 0.7045581177477463, 0.2916513759415957]",1
2826,OpenStax Biology 2e,10,3.34,"Visual Connection Figure 10.6 Karyokinesis (or mitosis) is divided into five stages—prophase, prometaphase, metaphase, anaphase, and telophase.","[Visual, Connection, , Figure, , 10.6, , Karyok